## DECISION TREE + DBUTILS.FS

In [0]:
# Definición de contexto
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [0]:
# Importamos librerías necesarias
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

In [0]:
# Definimos función para "etiquetar" la clase con un valor numérico.
def getLabelNum(label):
    if label == "Iris-setosa":
        return 0
    elif label == "Iris-versicolor":
        return 1
    else:
        return 2

NOTA:
En el contexto de *Apache Spark* y su biblioteca de aprendizaje automático *MLlib*, **LabeledPoint** es una estructura de datos utilizada específicamente para algoritmos de aprendizaje supervisado. Es esencialmente un contenedor que agrupa una etiqueta y un vector de características. Está diseñada para ser usada en problemas de clasificación y regresión donde cada instancia de datos tiene una etiqueta (o valor objetivo) y un conjunto de características numéricas.
**LabeledPoint** espera un valor numérico para la etiqueta y un vector numérico para las características. Si p[0], p[1], p[2], p[3] son strings (lo cual es muy probable si vienen directamente de un archivo CSV), será necesario convertirlos a float:

In [0]:
# Cargamos el dataset
data = (sc
        .textFile("/FileStore/tables/iris.csv")
        .map(lambda l: l.split(","))
        .filter(lambda p: len(p) == 5)  # Garantiza que solo se procesen líneas con 5 elementos
        .map(lambda p: LabeledPoint(getLabelNum(p[4]),[float(p[0]), float(p[1]), float(p[2]), float(p[3])]))
)

In [0]:
# Division en conjuntos de train y test
(train, test) = data.randomSplit([0.7, 0.3])

In [0]:
# Entrenamiento del modelo basado en árbol de decisión
model = DecisionTree.trainClassifier(train, 
                                     numClasses= 3, 
                                     categoricalFeaturesInfo={}, 
                                     impurity="gini", 
                                     maxDepth= 5, 
                                     maxBins= 32) 

In [0]:
# Evaluación del modelo
predictions = model.predict(test.map(lambda x: x.features))
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda vp: vp[0] != vp[1]).count() / float(test.count())
print ('Error en conjunto TEST = ' + str(testErr))

Error en conjunto TEST = 0.07317073170731707


In [0]:
print (model.toDebugString())

DecisionTreeModel classifier of depth 3 with 7 nodes
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 2 <= 4.95)
    If (feature 3 <= 1.65)
     Predict: 1.0
    Else (feature 3 > 1.65)
     Predict: 2.0
   Else (feature 2 > 4.95)
    Predict: 2.0



Es posible utilizar el directorio /dbfs/ para acceder al DBFS. Los caminos relativos como target/tmp/... deben ser convertidos en caminos absolutos

In [0]:
# Guardado y carga del modelo
model_path = "/FileStore/DecisionTreeClassificationModel"
model.save(sc, model_path)
sameModel = DecisionTreeModel.load(sc, model_path)

## Gestión de archivos subidos al WorkSpace del usuario

1. Listado de ficheros en el directorio raíz (/FileStore/tables)

In [0]:
%fs ls /FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/cancer_data.csv,cancer_data.csv,125204,1714381712000
dbfs:/FileStore/tables/data.csv,data.csv,4551,1714328543000
dbfs:/FileStore/tables/empleados.txt,empleados.txt,226,1713106514000
dbfs:/FileStore/tables/empleados2.txt,empleados2.txt,227,1713285254000
dbfs:/FileStore/tables/heart.csv,heart.csv,11323,1712559935000
dbfs:/FileStore/tables/iris.csv,iris.csv,4551,1714330197000
dbfs:/FileStore/tables/lung_cancer.csv,lung_cancer.csv,1704,1714388630000
dbfs:/FileStore/tables/pdi_sales.csv,pdi_sales.csv,38841198,1713111030000
dbfs:/FileStore/tables/purchases-1.csv,purchases-1.csv,25529445,1711916992000
dbfs:/FileStore/tables/purchases-2.csv,purchases-2.csv,25529445,1711952919000


In [0]:
# Listar archivos en un directorio específico - con función display y usando la API 'dbutils.fs'
display(dbutils.fs.ls("/FileStore/DecisionTreeClassificationModel/data/"))

path,name,size,modificationTime
dbfs:/FileStore/DecisionTreeClassificationModel/data/_SUCCESS,_SUCCESS,0,1716748783000
dbfs:/FileStore/DecisionTreeClassificationModel/data/_committed_5965025647390906940,_committed_5965025647390906940,816,1716748783000
dbfs:/FileStore/DecisionTreeClassificationModel/data/_started_5965025647390906940,_started_5965025647390906940,0,1716748777000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00000-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-72-1-c000.snappy.parquet,part-00000-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-72-1-c000.snappy.parquet,1733,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00001-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-73-1-c000.snappy.parquet,part-00001-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-73-1-c000.snappy.parquet,3813,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00002-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-74-1-c000.snappy.parquet,part-00002-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-74-1-c000.snappy.parquet,3543,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00003-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-75-1-c000.snappy.parquet,part-00003-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-75-1-c000.snappy.parquet,3813,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00004-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-76-1-c000.snappy.parquet,part-00004-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-76-1-c000.snappy.parquet,3814,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00005-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-77-1-c000.snappy.parquet,part-00005-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-77-1-c000.snappy.parquet,3543,1716748782000
dbfs:/FileStore/DecisionTreeClassificationModel/data/part-00006-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-78-1-c000.snappy.parquet,part-00006-tid-5965025647390906940-b7103748-d5f6-4856-90d6-4708e8c2a8b5-78-1-c000.snappy.parquet,3543,1716748782000


### Borrado de ficheros del WorkSpace de usuario

1. Usando comandos mágicos

In [0]:
%fs rm -r /FileStore/tables/data.csv

2. Usando la API **dbutils.fs** de la forma siguiente:

In [0]:
dbutils.fs.rm("/FileStore/tables/data.csv", recurse=True)

Aquí, *recurse=True* permite eliminar directorios y su contenido. Similar al comando *%fs rm*, usa esta opción con cuidado para evitar la eliminación accidental de datos importantes.

### Cambiar nombre a ficheros del WorkSpace de usuario

Hay que usar el comando **mv** (move). No existe el comando rename!!

1. Usando comandos mágicos

In [0]:
%fs mv dbfs:/FileStore/tables/nombre_actual.csv dbfs:/FileStore/tables/nuevo_nombre.csv

2. Usando la API **dbutils.fs**

In [0]:
dbutils.fs.mv("dbfs:/FileStore/tables/nombre_actual.csv", "dbfs:/FileStore/tables/nuevo_nombre.csv")

Más información sobre la API dbutils.fs en la siguiente URL:
https://docs.databricks.com/en/dev-tools/databricks-utils.html
